# T1 - Reprise du modèle défini à la partie 1

Notre modèle dimensionnel comprend :
- 2 tables de faits : FACT_SALES et FACT_INVENTORY
- 5 dimensions : DIM_TIME, DIM_PRODUCT, DIM_CUSTOMER, DIM_GEOGRAPHY, DIM_DEPARTMENT

Vérification de la faisabilité avec les données disponibles :
- Pour FACT_SALES : Nous avons orders.csv et order_details.csv qui contiennent les données nécessaires
- Pour FACT_INVENTORY : Nous devrons dériver cette table à partir des ventes et des capacités de stockage
- Pour les dimensions : Nous avons les fichiers correspondants (customers.csv, products.csv, etc.)

Le modèle est donc réalisable avec les données disponibles. Certaines mesures devront être calculées ou estimées.

# T2 - Importation des fichiers CSV et inspection des structures
Commençons par importer les fichiers CSV et examiner leur structure :

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

In [2]:
# Importation des fichiers CSV
addresses_df = pd.read_csv('data/addresses.csv')
customers_df = pd.read_csv('data/customers.csv')
departments_df = pd.read_csv('data/departments.csv')
order_details_df = pd.read_csv('data/order_details.csv')
orders_df = pd.read_csv('data/orders.csv')
products_df = pd.read_csv('data/products.csv')

In [3]:
# Inspection des structures
print("=== Aperçu de addresses_df ===")
print(addresses_df.info())
print(addresses_df.head())
print("\n=== Aperçu de customers_df ===")
print(customers_df.info())
print(customers_df.head())
print("\n=== Aperçu de departments_df ===")
print(departments_df.info())
print(departments_df.head())
print("\n=== Aperçu de order_details_df ===")
print(order_details_df.info())
print(order_details_df.head())
print("\n=== Aperçu de orders_df ===")
print(orders_df.info())
print(orders_df.head())
print("\n=== Aperçu de products_df ===")
print(products_df.info())
print(products_df.head())

=== Aperçu de addresses_df ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4910 entries, 0 to 4909
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Address ID   4910 non-null   object
 1   Customer ID  4910 non-null   object
 2   Country      4910 non-null   object
 3   City         4910 non-null   object
 4   State        4910 non-null   object
 5   Postal Code  4910 non-null   int64 
 6   Region       4910 non-null   object
dtypes: int64(1), object(6)
memory usage: 268.6+ KB
None
       Address ID Customer ID        Country             City           State  \
0  CG-12520_42420    CG-12520  United States        Henderson        Kentucky   
1  DV-13045_90036    DV-13045  United States      Los Angeles      California   
2  SO-20335_33311    SO-20335  United States  Fort Lauderdale         Florida   
3  BH-11710_90032    BH-11710  United States      Los Angeles      California   
4  AA-10480_28027    AA-10480  U

In [4]:
# Vérification des valeurs manquantes
print("\n=== Valeurs manquantes ===")
for df_name, df in [("addresses_df", addresses_df), 
                    ("customers_df", customers_df),
                    ("departments_df", departments_df),
                    ("order_details_df", order_details_df),
                    ("orders_df", orders_df),
                    ("products_df", products_df)]:
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        print(f"{df_name} contient des valeurs manquantes:")
        print(missing_values[missing_values > 0])
    else:
        print(f"{df_name} ne contient pas de valeurs manquantes")


=== Valeurs manquantes ===
addresses_df ne contient pas de valeurs manquantes
customers_df ne contient pas de valeurs manquantes
departments_df ne contient pas de valeurs manquantes
order_details_df ne contient pas de valeurs manquantes
orders_df ne contient pas de valeurs manquantes
products_df ne contient pas de valeurs manquantes


In [6]:
# Vérification des doublons
print("\n=== Doublons ===")
for df_name, df in [("addresses_df", addresses_df), 
                    ("customers_df", customers_df),
                    ("departments_df", departments_df),
                    ("order_details_df", order_details_df),
                    ("orders_df", orders_df),
                    ("products_df", products_df)]:
    duplicates = df.duplicated().sum()
    print(f"{df_name} contient {duplicates} doublons")


=== Doublons ===
addresses_df contient 0 doublons
customers_df contient 0 doublons
departments_df contient 0 doublons
order_details_df contient 0 doublons
orders_df contient 0 doublons
products_df contient 0 doublons


In [15]:
# Vérification des types de données
print("\n=== Types de données ===")
for df_name, df in [("addresses_df", addresses_df), 
                    ("customers_df", customers_df),
                    ("departments_df", departments_df),
                    ("order_details_df", order_details_df),
                    ("orders_df", orders_df),
                    ("products_df", products_df)]:
    print(f"{df_name} types:")
    print(df.dtypes)


=== Types de données ===
addresses_df types:
Address ID     object
Customer ID    object
Country        object
City           object
State          object
Postal Code     int64
Region         object
dtype: object
customers_df types:
Customer ID      object
Customer Name    object
Segment          object
dtype: object
departments_df types:
Department          object
City                object
Storage Capacity     int64
dtype: object
order_details_df types:
Order ID       object
Product ID     object
Sales         float64
Quantity        int64
Profit        float64
dtype: object
orders_df types:
Order ID               object
Order Date     datetime64[ns]
Ship Date      datetime64[ns]
Customer ID            object
Address ID             object
dtype: object
products_df types:
Product ID      object
Department      object
Sub-Category    object
Product Name    object
dtype: object


Après cette inspection, voici ce que nous pouvons relever :
- Contrairement à nos suppositions, aucun des dataframes ne contient de valeurs manquantes, ce qui signifie que les données sont d'excellente qualité.
- Aucun des dataframes ne contient de doublons, ce qui simplifie le processus de nettoyage.
- Les dates dans orders_df (Order Date, Ship Date) sont au format string (object) et doivent être converties en format datetime pour permettre des opérations temporelles.
- Les autres types de données semblent appropriés (les valeurs numériques sont déjà en int64 ou float64).

# T3 - Nettoyage des données
Maintenant, procédons au nettoyage des données pour chaque dataframe :

In [21]:
# Vérification et traitement des valeurs manquantes (il ne semble pas y en avoir, mais on ne sait jamais)
# Pour addresses_df
if addresses_df.isnull().sum().sum() > 0: # le premier .sum() donne le nombre de isNull par colonne, le second fait la somme de ces isNull
    # Remplir les valeurs manquantes ou supprimer les lignes selon le contexte
    addresses_df = addresses_df.dropna(subset=['Customer ID', 'City', 'State', 'Region'])
    # Pour les autres colonnes, on peut remplir avec des valeurs par défaut
    addresses_df['Postal Code'].fillna('Unknown', inplace=True)

# Pour customers_df
if customers_df.isnull().sum().sum() > 0:
    # Les informations client doivent être complètes pour être exploitables, donc on supprime les lignes avec des valeurs manquantes
    customers_df = customers_df.dropna()

# Pour order_details_df
if order_details_df.isnull().sum().sum() > 0:
    # Vérifier les colonnes avec des valeurs manquantes
    # Pour les valeurs numériques, on peut remplacer par 0 ou la moyenne
    order_details_df['Quantity'].fillna(0, inplace=True)
    order_details_df['Sales'].fillna(0, inplace=True)
    order_details_df['Profit'].fillna(0, inplace=True)
    # Supprimer les lignes où Product ID est manquant
    order_details_df = order_details_df.dropna(subset=['Product ID'])

# Pour products_df
if products_df.isnull().sum().sum() > 0:
    # Les informations produit sont essentielles, donc on supprime les lignes avec des valeurs manquantes
    products_df = products_df.dropna()

In [22]:
# Suppression des doublons (les outputs de T2 n'en montrent pas, mais on ne sait jamais)
addresses_df = addresses_df.drop_duplicates()
customers_df = customers_df.drop_duplicates()
departments_df = departments_df.drop_duplicates()
order_details_df = order_details_df.drop_duplicates()
orders_df = orders_df.drop_duplicates()
products_df = products_df.drop_duplicates()

In [23]:
# Conversion des dates dans orders_df
orders_df['Order Date'] = pd.to_datetime(orders_df['Order Date'])
orders_df['Ship Date'] = pd.to_datetime(orders_df['Ship Date'])

In [24]:
# Normalisation (nettoyage et uniformisation) des valeurs textuelles
# Pour products_df
products_df['Department'] = products_df['Department'].str.strip().str.title()
products_df['Sub-Category'] = products_df['Sub-Category'].str.strip().str.title()
products_df['Product Name'] = products_df['Product Name'].str.strip()

# Pour customers_df
customers_df['Segment'] = customers_df['Segment'].str.strip().str.title()
customers_df['Customer Name'] = customers_df['Customer Name'].str.strip()

# Pour addresses_df
addresses_df['City'] = addresses_df['City'].str.strip().str.title()
addresses_df['State'] = addresses_df['State'].str.strip().str.title()
addresses_df['Region'] = addresses_df['Region'].str.strip().str.title()
addresses_df['Country'] = addresses_df['Country'].str.strip().str.title()

# Pour departments_df
departments_df['Department'] = departments_df['Department'].str.strip().str.title()
departments_df['City'] = departments_df['City'].str.strip().str.title()

In [25]:
# Vérification des données après nettoyage
print("\n=== Données après nettoyage ===")
for df_name, df in [("addresses_df", addresses_df), 
                    ("customers_df", customers_df),
                    ("departments_df", departments_df),
                    ("order_details_df", order_details_df),
                    ("orders_df", orders_df),
                    ("products_df", products_df)]:
    print(f"{df_name} shape: {df.shape}")
    print(f"Valeurs manquantes: {df.isnull().sum().sum()}")
    print(f"Doublons: {df.duplicated().sum()}")


=== Données après nettoyage ===
addresses_df shape: (4910, 7)
Valeurs manquantes: 0
Doublons: 0
customers_df shape: (793, 3)
Valeurs manquantes: 0
Doublons: 0
departments_df shape: (3, 3)
Valeurs manquantes: 0
Doublons: 0
order_details_df shape: (9993, 5)
Valeurs manquantes: 0
Doublons: 0
orders_df shape: (5009, 5)
Valeurs manquantes: 0
Doublons: 0
products_df shape: (1862, 4)
Valeurs manquantes: 0
Doublons: 0


# T4 - Réalisation des jointures nécessaires
Maintenant, créons les tables dimensionnelles et les tables de faits conformément à notre modèle en étoile :

In [26]:
# Création de DIM_TIME
def create_dim_time(orders_df):
    # Extraire toutes les dates uniques des commandes et expéditions
    all_dates = pd.concat([orders_df['Order Date'], orders_df['Ship Date']]).drop_duplicates().sort_values()
    
    # Créer le dataframe de dimension temporelle
    dim_time = pd.DataFrame({
        'date': all_dates,
        'day_of_week': all_dates.dt.day_name(),
        'month': all_dates.dt.month_name(),
        'quarter': all_dates.dt.quarter,
        'year': all_dates.dt.year
    })
    
    # Ajouter la saison
    def get_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Fall'
    
    dim_time['season'] = dim_time['date'].dt.month.apply(get_season)
    
    # Ajouter une clé primaire
    dim_time['time_id'] = range(1, len(dim_time) + 1)
    
    return dim_time

# Création de DIM_PRODUCT
def create_dim_product(products_df):
    # Copier les données produits
    dim_product = products_df.copy()
    
    # Renommer les colonnes pour correspondre au modèle
    dim_product = dim_product.rename(columns={
        'Product ID': 'product_id',
        'Product Name': 'product_name',
        'Sub-Category': 'sub_category',
        'Department': 'department'
    })
    
    return dim_product

# Création de DIM_CUSTOMER
def create_dim_customer(customers_df, orders_df):
    # Copier les données clients
    dim_customer = customers_df.copy()
    
    # Calculer la date de première commande pour chaque client
    first_orders = orders_df.groupby('Customer ID')['Order Date'].min().reset_index()
    first_orders = first_orders.rename(columns={'Order Date': 'first_order_date'})
    
    # Calculer le nombre de commandes par client
    order_counts = orders_df.groupby('Customer ID').size().reset_index(name='order_count')
    
    # Joindre ces informations à la dimension client
    dim_customer = dim_customer.merge(first_orders, on='Customer ID', how='left')
    dim_customer = dim_customer.merge(order_counts, on='Customer ID', how='left')
    
    # Renommer les colonnes pour correspondre au modèle
    dim_customer = dim_customer.rename(columns={
        'Customer ID': 'customer_id',
        'Customer Name': 'customer_name',
        'Segment': 'segment'
    })
    
    return dim_customer

# Création de DIM_GEOGRAPHY
def create_dim_geography(addresses_df):
    # Copier les données d'adresses
    dim_geography = addresses_df.copy()
    
    # Renommer les colonnes pour correspondre au modèle
    dim_geography = dim_geography.rename(columns={
        'Address ID': 'geography_id',
        'City': 'city',
        'State': 'state',
        'Region': 'region',
        'Country': 'country',
        'Postal Code': 'postal_code'
    })
    
    # Supprimer la colonne Customer ID qui n'est pas nécessaire dans cette dimension
    dim_geography = dim_geography.drop(columns=['Customer ID'])
    
    return dim_geography

# Création de DIM_DEPARTMENT
def create_dim_department(departments_df):
    # Copier les données de départements
    dim_department = departments_df.copy()
    
    # Renommer les colonnes pour correspondre au modèle
    dim_department = dim_department.rename(columns={
        'Department': 'department_name',
        'City': 'city',
        'Storage Capacity': 'storage_capacity'
    })
    
    # Ajouter une clé primaire
    dim_department['department_id'] = range(1, len(dim_department) + 1)
    
    return dim_department

In [27]:
# Création de FACT_SALES
def create_fact_sales(orders_df, order_details_df, dim_time, dim_customer, dim_product, dim_geography):
    # Joindre les commandes et les détails de commande
    fact_sales = orders_df.merge(order_details_df, on='Order ID', how='inner')
    
    # Calculer le délai de livraison en jours
    fact_sales['delivery_time'] = (fact_sales['Ship Date'] - fact_sales['Order Date']).dt.days
    
    # Créer des mappings pour les clés étrangères
    # Pour la dimension temps, nous avons besoin de mapper les dates de commande
    time_mapping = pd.Series(dim_time['time_id'].values, index=dim_time['date']).to_dict()
    
    # Ajouter les clés étrangères
    fact_sales['time_id'] = fact_sales['Order Date'].map(time_mapping)
    
    # Renommer les colonnes pour correspondre au modèle
    fact_sales = fact_sales.rename(columns={
        'Order ID': 'order_id',
        'Customer ID': 'customer_id',
        'Product ID': 'product_id',
        'Address ID': 'geography_id',
        'Quantity': 'quantity',
        'Sales': 'sales',
        'Profit': 'profit'
    })
    
    # Sélectionner uniquement les colonnes nécessaires
    fact_sales = fact_sales[['order_id', 'time_id', 'customer_id', 'product_id', 'geography_id', 
                             'quantity', 'sales', 'profit', 'delivery_time']]
    
    return fact_sales

# Création de FACT_INVENTORY
def create_fact_inventory(fact_sales, dim_product, dim_department, dim_time):
    # Agréger les ventes par produit et par période
    sales_agg = fact_sales.groupby(['time_id', 'product_id']).agg({
        'quantity': 'sum'
    }).reset_index()
    
    # Créer un mapping entre les produits et leurs départements
    product_dept_mapping = dim_product.set_index('product_id')['department'].to_dict()
    
    # Ajouter le département à chaque ligne
    sales_agg['department'] = sales_agg['product_id'].map(product_dept_mapping)
    
    # Créer un mapping entre les noms de département et leurs IDs
    dept_id_mapping = dim_department.set_index('department_name')['department_id'].to_dict()
    
    # Ajouter l'ID du département
    sales_agg['department_id'] = sales_agg['department'].map(dept_id_mapping)
    
    # Créer un mapping pour la capacité de stockage par département
    storage_capacity_mapping = dim_department.set_index('department_id')['storage_capacity'].to_dict()
    
    # Ajouter la capacité de stockage
    sales_agg['storage_capacity'] = sales_agg['department_id'].map(storage_capacity_mapping)
    
    # Calculer un taux d'utilisation estimé (simplifié)
    # Nous utilisons la quantité vendue divisée par la capacité de stockage comme approximation
    sales_agg['utilization_rate'] = sales_agg['quantity'] / sales_agg['storage_capacity']
    
    # Renommer pour correspondre au modèle
    fact_inventory = sales_agg.rename(columns={
        'quantity': 'quantity_sold'
    })
    
    # Ajouter un ID unique
    fact_inventory['inventory_id'] = range(1, len(fact_inventory) + 1)
    
    # Sélectionner uniquement les colonnes nécessaires
    fact_inventory = fact_inventory[['inventory_id', 'time_id', 'product_id', 'department_id', 
                                     'quantity_sold', 'storage_capacity', 'utilization_rate']]
    
    return fact_inventory

In [28]:
# Création des dimensions
dim_time = create_dim_time(orders_df)
dim_product = create_dim_product(products_df)
dim_customer = create_dim_customer(customers_df, orders_df)
dim_geography = create_dim_geography(addresses_df)
dim_department = create_dim_department(departments_df)

# Création des tables de faits
fact_sales = create_fact_sales(orders_df, order_details_df, dim_time, dim_customer, dim_product, dim_geography)
fact_inventory = create_fact_inventory(fact_sales, dim_product, dim_department, dim_time)

# Vérification des tables créées
print("\n=== Tables dimensionnelles créées ===")
print(f"DIM_TIME: {dim_time.shape}")
print(f"DIM_PRODUCT: {dim_product.shape}")
print(f"DIM_CUSTOMER: {dim_customer.shape}")
print(f"DIM_GEOGRAPHY: {dim_geography.shape}")
print(f"DIM_DEPARTMENT: {dim_department.shape}")

print("\n=== Tables de faits créées ===")
print(f"FACT_SALES: {fact_sales.shape}")
print(f"FACT_INVENTORY: {fact_inventory.shape}")

# Afficher un aperçu des tables
print("\n=== Aperçu de DIM_TIME ===")
print(dim_time.head())
print("\n=== Aperçu de FACT_SALES ===")
print(fact_sales.head())
print("\n=== Aperçu de FACT_INVENTORY ===")
print(fact_inventory.head())


=== Tables dimensionnelles créées ===
DIM_TIME: (1434, 7)
DIM_PRODUCT: (1862, 4)
DIM_CUSTOMER: (793, 5)
DIM_GEOGRAPHY: (4910, 6)
DIM_DEPARTMENT: (3, 4)

=== Tables de faits créées ===
FACT_SALES: (9993, 9)
FACT_INVENTORY: (9955, 7)

=== Aperçu de DIM_TIME ===
           date day_of_week     month  quarter  year  season  time_id
3952 2018-12-28      Friday  December        4  2018  Winter        1
343  2018-12-29    Saturday  December        4  2018  Winter        2
854  2018-12-30      Sunday  December        4  2018  Winter        3
2647 2018-12-31      Monday  December        4  2018  Winter        4
3796 2019-01-01     Tuesday   January        1  2019  Winter        5

=== Aperçu de FACT_SALES ===
         order_id  time_id customer_id       product_id    geography_id  \
0  CA-2016-152156     1011    CG-12520  FUR-BO-10001798  CG-12520_42420   
1  CA-2016-152156     1011    CG-12520  FUR-CH-10000454  CG-12520_42420   
2  CA-2016-138688      863    DV-13045  OFF-LA-10000240  DV-1304

# T5 - Enregistrement des tables finales au format CSV
Maintenant, enregistrons toutes nos tables au format CSV pour pouvoir les importer dans Power BI :

In [29]:
# Création d'un dossier pour stocker les fichiers si nécessaire
import os
if not os.path.exists('output_tables'):
    os.makedirs('output_tables')

# Enregistrement des tables dimensionnelles
dim_time.to_csv('output_tables/dim_time.csv', index=False)
dim_product.to_csv('output_tables/dim_product.csv', index=False)
dim_customer.to_csv('output_tables/dim_customer.csv', index=False)
dim_geography.to_csv('output_tables/dim_geography.csv', index=False)
dim_department.to_csv('output_tables/dim_department.csv', index=False)

# Enregistrement des tables de faits
fact_sales.to_csv('output_tables/fact_sales.csv', index=False)
fact_inventory.to_csv('output_tables/fact_inventory.csv', index=False)

print("Toutes les tables ont été enregistrées dans le dossier 'output_tables'")

Toutes les tables ont été enregistrées dans le dossier 'output_tables'
